<a href="https://colab.research.google.com/github/k0inwork/colabs/blob/main/Json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cp /content/drive/MyDrive/task_backend_developer_november_2020.zip .
!unzip -o  task*.zip


Archive:  task_backend_developer_november_2020.zip
  inflating: task_folder/event/fb1a0854-9535-404d-9bdd-9ec0abb6cd6c.json  
  inflating: task_folder/event/3ade063d-d1b9-453f-85b4-dda7bfda4711.json  
  inflating: task_folder/event/3b4088ef-7521-4114-ac56-57c68632d431.json  
  inflating: task_folder/event/ffe6b214-d543-40a8-8da3-deb0dc5bbd8c.json  
  inflating: task_folder/event/e2d760c3-7e10-4464-ab22-7fda6b5e2562.json  
  inflating: task_folder/event/ba25151c-914f-4f47-909a-7a65a6339f34.json  
  inflating: task_folder/event/1eba2aa1-2acf-460d-91e6-55a8c3e3b7a3.json  
  inflating: task_folder/event/cc07e442-7986-4714-8fc2-ac2256690a90.json  
  inflating: task_folder/event/f5656ff6-29e1-46b0-8d8a-ff77f9cc0953.json  
  inflating: task_folder/event/297e4dc6-07d1-420d-a5ae-e4aff3aedc19.json  
  inflating: task_folder/event/bb998113-bc02-4cd1-9410-d9ae94f53eb0.json  
  inflating: task_folder/event/2e8ffd3c-dbda-42df-9901-b7a30869511a.json  
  inflating: task_folder/event/6b1984e5-4092-4279

In [ ]:
import os,json, jsonschema
import logging
#logger = logging.getLogger()
#logger.setLevel(logging.NOTSET)
#logger.setLevel(logging.ERROR)
logger.setLevel(logging.CRITICAL)


#loads a json file 
def loadJson(filename):
  with open(filename) as f:
    try:
      j = json.load(f)
    except ValueError as err:
      logger.error("File "+filename+" not found or is not a valid json file",exc_info=err)
      return None
  return j

#validates schema
def validateSchema(json=j,schema=schema,filename=filename):
  if schema is not None:
    try:
      jsonschema.validate(j,schema)
    except jsonschema.ValidationError as err:
      logger.error('For file '+filename+' schema is not valid.\n'+err.message)
      return False
    except Exception as err:
      logger.error('For file '+filename+' during validation error occured.',exc_info=err)
      return False
  
  logger.log(msg="Valid.",level=logging.INFO)
  return True

In [ ]:
categories = [] 
catschemas = {}
wrong=[]

for cat_ in os.listdir('task_folder/schema'):
  cat = cat_.split('.')[0]
  filename = 'task_folder/schema/'+cat_
  logging.log(msg="Loading schema for category "+cat+" from file "+filename,level=logging.INFO)
  j = loadJson(filename)
  if j is not None:
    catschemas[cat]=j
    categories.append(cat)
  else: 
    wrong.append((cat_, 'Category not loaded from file - possible wrong json'))

In [ ]:

dir = 'task_folder/event'

all = {a:[] for a in categories}
good = {a:[] for a in categories}

#wrong = []

for file_ in os.listdir(dir):
  filename = dir+'/'+file_
  j = loadJson(filename)

  if j is None:
    wrong.append((file_,"Not loaded (could be wrong json)"))
 
  else:

    if j.get('event') is None:
      logger.error("No event for file "+filename+" provided.")
      wrong.append((file_,"No event category in the json"))
    else:
      event = j['event']
      logger.log(msg="Event is "+event,level=logging.INFO)

      if event in categories:
        all[event].append((file_,j))
        logger.log(msg="Validating schema for event",level=logging.INFO)
        if validateSchema(json=j,schema=catschemas[event],filename=file_):
          good[event].append((file_,j))
      else:
        wrong.append((file_,"No schema for such event category (category : "+event+")"))


In [ ]:
#search wrong schemas
wrong_schemas = [i for i in categories if len(good[i])==0 and len(all[i])>0 ]

for w_s in wrong_schemas:
  wrong.append((w_s+".schema","No json passed the schema - could be wrong schema instead"))

#add unvalidated events for good schemas
good_schemas = [s for s in categories if s not in wrong_schemas]

for s in good_schemas:
  bad_events = [ev for ev in all[s] if ev not in good[s]]
  for ev in bad_events:
    wrong.append((ev[0],"Did not pass validation(schema : "+s+")"))


Result of work of the script is below

In [ ]:
import pandas as pd

pd.DataFrame(columns=['File name with problem','Description'],data=[{'File name with problem':a,'Description':b} for (a,b) in wrong])

#wrong

,File name with problem,Description
0,2e8ffd3c-dbda-42df-9901-b7a30869511a.json,No schema for such event category (category : ...
1,29f0bfa7-bd51-4d45-93be-f6ead1ae0b96.json,Not loaded (could be wrong json)
2,a95d845c-8d9e-4e07-8948-275167643a40.json,No event category in the json
3,6b1984e5-4092-4279-9dce-bdaa831c7932.json,No schema for such event category (category : ...
4,ba25151c-914f-4f47-909a-7a65a6339f34.json,No schema for such event category (category : ...
5,3ade063d-d1b9-453f-85b4-dda7bfda4711.json,No schema for such event category (category : ...
6,c72d21cf-1152-4d8e-b649-e198149d5bbb.json,No schema for such event category (category : ...
7,label_selected.schema,No json passed the schema - could be wrong sch...
8,cmarker_created.schema,No json passed the schema - could be wrong sch...
9,sleep_created.schema,No json passed the schema - could be wrong sch...
